In [1]:
import matplotlib
import matplotlib.pyplot as plt

import os
import pathlib
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

# Helper functions

In [2]:
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + model_file,
                                        untar=True)
    return str(model_dir)

MODEL_DATE = '20200711'
MODEL_NAME = 'centernet_hg104_1024x1024_coco17_tpu-32'
# PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)
PATH_TO_MODEL_DIR='C:\\Users\\Roman\\.keras\\datasets\\centernet_hg104_1024x1024_coco17_tpu-32'

In [3]:
# Download labels file
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
# PATH_TO_LABELS = download_labels(LABEL_FILENAME)
PATH_TO_LABELS='C:\\Users\\Roman\\.keras\\datasets\\mscoco_label_map.pbtxt'

In [4]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Load Model

In [5]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL);

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

Done! Took 122.17579650878906 seconds


# Load label map data

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

# Run BEAMNG

In [7]:
%matplotlib
import mmap
import random
import sys
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from time import sleep
import numpy as np
from beamngpy import BeamNGpy, Scenario, Vehicle, setup_logging
from beamngpy.sensors import Camera, Timer, Lidar

Using matplotlib backend: TkAgg


In [8]:
# fig = plt.figure(1, figsize=(10, 5))
# axarr = fig.subplots(1, 1)

# a_colour = axarr

In [16]:
car_pos = (-769.1, 400.8, 142.8)
pos = (-0.3, 1, 1.0)
direction = (0, 1, 0)
fov = 90
resolution = (512, 512)
front_camera = Camera(pos, direction, fov, resolution,
                          colour=True)

In [17]:
# setup_logging()

beamng = BeamNGpy('localhost', 64256, home='E:/BeamNG.research')

scenario = Scenario('west_coast_usa', 'example')

vehicle = Vehicle('ego_vehicle', model='etk800',
              licence='RED', color='Red')
timer = Timer()

In [18]:
vehicle.attach_sensor('front_camera', front_camera)
vehicle.attach_sensor('timer', timer)
scenario.add_vehicle(vehicle, pos=car_pos, rot=None, rot_quat=(0, 0, 0.3826834, 0.9238795))
scenario.make(beamng)

bng = beamng.open(launch=True)

In [19]:
images = []

In [20]:
# plt.ion()

try:
    bng.set_deterministic()  # Set simulator to be deterministic
    bng.set_steps_per_second(60)  # With 60hz temporal resolution

    bng.load_scenario(scenario)
    bng.hide_hud()
    bng.start_scenario()
    vehicle.ai_set_mode('span')
    #print('Driving around for 60 seconds...')
    #bng.step(50)
    bng.pause()
  
#     for _ in range(50):
#         bng.step(20)

#         Retrieve sensor data and show the camera data.
    sensors = bng.poll_sensors(vehicle)
    images.append(sensors['front_camera']['colour'].convert('RGB'))

#         print('{} seconds passed.'.format(sensors['timer']['time']))

#         a_colour.imshow(sensors['front_camera']['colour'].convert('RGB'))
#         plt.pause(0.00001)
finally:
    bng.close()

In [14]:
len(images)

1

# Run Detector

In [21]:
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings




for image in images:

    image_np = load_image_into_numpy_array(image)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()

# sphinx_gallery_thumbnail_number = 2

Done


# Evaluation